### 2. - Limpeza dos dados (Data Cleanning)

In [73]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import datetime
import re


#----------------------------------------------------------------------------------
#  CIDADE / CITY
#----------------------------------------------------------------------------------
map_cidade = {"Campo0 Limpo Paulista" : "Campo Limpo Paulista",
              "Jundiai" : "Jundiaí",
              "JundiaÌ" : "Jundiaí",
              "Jundiái" : "Jundiaí",
              "JUndiaí" : "Jundiaí",
              "Mairipora" : "Mairiporã",
              "Mariporã"  : "Mairiporã",
              "Várzea Paulista, SP" : "Várzea Paulista"}

def mapping_city_name(city_name, mapping):
    return mapping.get(city_name, None)          
    
def is_city_name(element):
    return (element.tag == "tag") and (element.attrib['k'] == "addr:city")

def clean_city(osmfile, output):
    osm_file = open(osmfile, "r", encoding="utf-8")    
    context = ET.iterparse(osm_file, events=("start",))
    
    for event, element in context:
        if is_city_name(element):
            new_city = None
            new_city = mapping_city_name(element.attrib['v'], map_cidade)
            if new_city is not None:
                print(element.attrib['v'], " => ", new_city)
                element.attrib['v'] = new_city
    
    print("\n")            

    # Gravar correções realizadas em um novo arquivo
    first_line = '''<?xml version="1.0"?>'''+'\n'
    root = ET.tostring(context.root).decode("utf-8")    
    new_xml = first_line + root
    f = open(output, "w", encoding="utf-8")
    f.write(new_xml)
    f.close()    
    
    osm_file.close()






#----------------------------------------------------------------------------------
#  TELEFONE / PHONE
#----------------------------------------------------------------------------------

"""
https://wiki.openstreetmap.org/wiki/Key:phone#Usage
"""


phone_format_num1 = re.compile(r'^(... .. .... ....)|(... .. ..... ....)|(.... ... ....)')
phone_format_num2 = re.compile(r'^(... .. ....-....)|(... .. .....-....)')
problemchars = re.compile(r'([=\/&<>;\'"\?%#$@\,\.\t\r\n]|[A-z])')


def update_phone(phone):
    phone_temp = phone
    phone_temp = phone_temp.replace(":"," ",1)
    phone_temp = phone_temp.strip()
    phone_split = None

    if phone_temp.find(';') == -1:

        count = 0

        if phone_temp[0:5] == "(11) ":
            phone_temp = phone_temp.replace("(11) ", "+55 11 ", 1) # Substitui somente o primeiro que encontrar
            
        if phone_temp[0:3] == "11 ":
            phone_temp = phone_temp.replace("11 ", "+55 11 ", 1) # Substitui somente o primeiro que encontrar
            
        if phone_temp[0:3] == "55 ":
            phone_temp = phone_temp.replace("55 ", "+55 ", 1)
            
        if phone_temp[0:5] == "+ 55 ":
            phone_temp = phone_temp.replace("+ 55 ", "+55 ", 1)

        if len(phone_temp[3:4].strip()) == 0 and \
           len(phone_temp[6:7].strip()) == 0 and \
            phone_temp[11:12] != " " and phone_temp[11:12] != "-":
            phone_temp = phone_temp[0:11] + " " + phone_temp[11:]

        if audit_characters_phone(phone_temp) == False:
            print("Ainda inválido", phone_temp)
    
    else:
        phone_split = phone_temp.split(";")
        phone_temp = ""
       
        for x in phone_split:

            if x[0:5] == "(11) ":
                x = x.replace("(11) ", "+55 11 ", 1) # Substitui somente o primeiro que encontrar
                #break    
            if x[0:3] == "11 ":
                x = x.replace("11 ", "+55 11 ", 1) # Substitui somente o primeiro que encontrar
                #break
            if x[0:3] == "55 ":
                x = x.replace("55 ", "+55 ", 1)
                #break
            if x[0:5] == "+ 55 ":
                x = x.replace("+ 55 ", "+55 ", 1)

            if len(x[3:4].strip()) == 0 and \
               len(x[6:7].strip()) == 0 and \
               x[11:12] != " " and x[11:12] != "-":
               x =x[0:11] + " " + x[11:]
                #break       

            if audit_characters_phone(x) == False:
                print("Ainda inválido", x)
    
            phone_temp = phone_temp + x + ";" 
    
    if phone_split is not None:
        phone_temp = phone_temp[0:-1]
    
    return phone_temp


def audit_characters_phone(element):
    if element.find(';') == -1:
        classificated = False
        result = False

        # PROBLEMCHAR
        if (problemchars.search(element) is not None) and \
           (problemchars.search(element).group() != "+"):
            classificated = True        
            print(element, ' = classif.: PROBLEMCHAR')                

        # CORRETO num1
        if not classificated:
            if phone_format_num1.search(element) is not None:        
                result = True
                classificated = True        

        # num2
        if not classificated:
            if phone_format_num2.search(element) is not None:        
                result = True
                classificated = True        

        # OUTROS
        if not classificated:
            result = False      

    else:    
        classificated = False
        result = False
        
        elements = element.split(";")
        
        for x in elements:

            if problemchars.search(element) is not None:
                classificated = True        

            # CORRETO num1
            if not classificated:
                if phone_format_num1.search(element) is not None:        
                    result = True
                    classificated = True        

            # num2
            if not classificated:
                if phone_format_num2.search(element) is not None:        
                    result = True
                    classificated = True        

            # OUTROS
            if not classificated:
                result = False      
                
            if result == False:
                break

    
    return result


def clean_phone(osmfile, output):
    osm_file = open(osmfile, "r+", encoding="utf8")

    context = ET.iterparse(osm_file, events=("start",))
    
    for event, elem in context:
        if elem.tag == "node" or elem.tag == "way":
            
            for tag in elem.iter("tag"):
                if (tag.attrib['k'] == "phone"):
                    if audit_characters_phone(tag.attrib['v']) == False:
                        new_phone = update_phone(tag.attrib['v'])
                        print(tag.attrib['v'], " => ", new_phone)
                        tag.attrib['v'] = new_phone

    print("\n")            
        
    # Gravar correções realizadas em um novo arquivo
    first_line = '''<?xml version="1.0"?>'''+'\n'
    root = ET.tostring(context.root).decode("utf-8")    
    new_xml = first_line + root
    f = open(output, "w", encoding="utf-8")
    f.write(new_xml)
    f.close()    
    
    osm_file.close()




#----------------------------------------------------------------------------------
#  CÓDIGO POSTAL / POST CODE
#----------------------------------------------------------------------------------

accept_pc1 = re.compile(r'(.....?-...)')
accept_pc2 = re.compile(r'(........)')
problemchars = re.compile(r'([=\+/&<>;\'"\?%#$@\,\.\t\r\n]|[A-z])')

map_postal_code = {"3221-390" : "13221-390"}

def is_postal_code(elem):
    return ((elem.attrib['k'] == "addr:postcode") | (elem.attrib['k'] == "postal_code"))

def mapping_postal_code(postal_code, mapping):
    return mapping.get(postal_code, None)  

def audit_characters_postal_code(element):
    classificated = False
    result = False

    if len(element.replace("-","")) < 8:
        result = False
        classificated = True        
    
    # PROBLEMCHAR
    if problemchars.search(element) is not None:
        result = False
        classificated = True        
        
    # CORRETO 2
    if classificated == False:
        if accept_pc1.search(element) is not None:        
            result = True
            classificated = True   
            
    # CORRETO 2
    if classificated == False:
        if accept_pc2.search(element) is not None:        
            result = False
            classificated = True        
            
    if len(element) > 9:
        result = False
        
    
    # OUTROS
    if classificated == False:
        result = False                         
    
    return result

def update_postal_code(postal_code):
    
    if postal_code.find("-") == -1:    
        if postal_code[5:6] != "-":
            postal_code = postal_code[:5] + "-" + postal_code[5:]
    else:
        new_postal_code = None
        new_postal_code = mapping_postal_code(postal_code, map_postal_code)
        if new_postal_code is not None:
            postal_code = new_postal_code

    if audit_characters_postal_code(postal_code) == False:
        print("Código Postal ainda com problema: ", postal_code)
    
    return postal_code


def clean_post_code(osmfile, output):
    osm_file = open(osmfile, "r")
    
    context = ET.iterparse(osm_file, events=("start",))
    
    for event, elem in context:
        
        if elem.tag == "node" or elem.tag == "way":          
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    if audit_characters_postal_code(tag.attrib['v']) == False:
                        new_post_code = update_postal_code(tag.attrib['v'])
                        print(tag.attrib['v'], " => ", new_post_code)
                        tag.attrib['v'] = new_post_code
                            
    print("\n")     
    
    # Gravar correções realizadas em um novo arquivo
    first_line = '''<?xml version="1.0"?>'''+'\n'
    root = ET.tostring(context.root).decode("utf-8")    
    new_xml = first_line + root
    f = open(output, "w", encoding="utf-8")
    f.write(new_xml)
    f.close()    
    
    osm_file.close()
    
    
    
    
    
#----------------------------------------------------------------------------------
#  RUAS / STREETS
#----------------------------------------------------------------------------------    
    
accept = re.compile(r'^\w*$')
upper = re.compile(r'^([A-Z])*$')
lower_and_upper = re.compile(r'^([A-z]| )*$')
problemchars = re.compile(r'[=\+/&<>;"\?%#$@\,\t\r\n]')
with_number = re.compile(r'^[0-9]')

# O que for diferente destes tipos, devem obrigatoriamente ser verificados
expected = ["Rua", "Avenida", "Rodovia", "Travessa", "Viaduto", "Alameda", "Praça", "Via", "Estrada",
            "Viela", "Calçadão", "Largo", "Rodoanel", "Rotatória", "Alameda", "Caminho", "Corredor",
            "Passagem", "Acesso", "Marginal", "Trevo", "Passarela", "Contorno", "Travessia",
            "Paço", "Entrada", "Túnel", "Entrada", "Travessia", "Saída", "Alamedas", "Vila", "Trilha"]


map_street = { "rua" : "Rua",
               "R."  : "Rua",
               "Rod.": "Rodovia",
               "Rd"  : "Rodovia",
               "Av.enida" : "Avenida",
               "Av." : "Avenida",
               "Av"  : "Avenida",
               "Avenidas" : "Avenida",
               "estrada" : "Estrada",
               "AvenidaAssembléia" : "Avenida Assembléia",
               "Rod.Manoel" : "Rodovia Manoel"
               }

def audit_street(street_name, mapping):
    return mapping.get(street_name, None)     

def audit_street_type(street_types, street_name):
    result = False
    
    street_type = street_name.split()[0]
    
    if street_type == "RuaJuscelino":
        print("RuaJuscelino1")
    if street_type not in expected:
        if street_type[0:3] != "Rua":
            if street_type == "RuaJuscelino":
                print("RuaJuscelino2")               
            street_types[street_type].add(street_type)
            result = False
        else:
            if street_type == "RuaJuscelino":
                print("RuaJuscelino3")            
            result = True
    else:
        result = True

    return result
    
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")



def audit_characters_street(element):
    result = False
    classificated = False
    
    # UPPER
    if upper.search(element) is not None:
        classificated = True
        result = False
    
    # LOWER
    if not classificated:    
        if accept.search(element) is not None:
            result = True
            classificated = True

    
    # LOWER AND UPPER
    if not classificated:
        if lower_and_upper.search(element) is not None:
            result = True
            classificated = True        
            
    # with_number
    if not classificated:
        if with_number.search(element) is not None:        
            result = True
            classificated = True        

    # PROBLEMCHAR
    if problemchars.search(element) is not None:
        classificated = True
        result = False
        
    # OUTROS
    if not classificated:
        result = True
    
    return result

def update_street(street_name):
    street_name = street_name.replace("\" ", "")
    street_name = street_name.replace("\"", "")
    street_name = street_name.replace(",", "")
    
    street_type = street_name.split()[0]
    
    if (street_name[0:3] == "Rua") and (street_name[3:4] != " "):
        street_name = street_name[0:3] + " " + street_name[3:]
        print("Jucelino street name: ", street_name)
        
    # aqui
    new_street = None
    new_street = audit_street(street_type, map_street)
    if new_street is not None:
        street_name = street_name.replace(street_type, new_street)
    
    return street_name

def clean_street(osmfile, output):
    osm_file = open(osmfile, "r", encoding="utf-8")
    street_types = defaultdict(set)
    id_tag_highway = []
    
    context = ET.iterparse(osm_file, events=("start",))
    
    for event, elem in context:
        
        if elem.tag == "node" or elem.tag == "way":

            # Se a tag for WAY, deve-ser haver a subtag k = HIGHWAY para ser considerado do tipo Rua 

            if elem.tag == "way":
                for tag in elem.iter("tag"):
                    if tag.attrib['k'] == "highway":
                        if elem.attrib["id"] not in id_tag_highway:
                            id_tag_highway.append(elem.attrib["id"])
                        break
            
            # Se não estiver entre os tipos de ruas esperados, imprimir. Caso contrário, validar os caracteres
            for tag in elem.iter("tag"):
                if (elem.tag == "way" and tag.attrib['k'] == "name"):
                    if elem.attrib["id"] in id_tag_highway:
                        if audit_street_type(street_types, tag.attrib['v']):
                            if audit_characters_street(tag.attrib['v']) == False:
                                new_street = None
                                new_street = update_street(tag.attrib['v'])
                                print(tag.attrib['v'], " => ", new_street)
                                tag.attrib['v'] = new_street
                        else:
                            new_street = None
                            new_street = update_street(tag.attrib['v'])
                            print(tag.attrib['v'], " => ", new_street)
                            tag.attrib['v'] = new_street

                elif (elem.tag == "node" and tag.attrib['k'] == "addr:street"):
                    if audit_street_type(street_types, tag.attrib['v']):
                        if audit_characters_street(tag.attrib['v']) == False:
                            new_street = None
                            new_street = update_street(tag.attrib['v'])
                            print(tag.attrib['v'], " => ", new_street)
                            tag.attrib['v'] = new_street
                    else:
                        new_street = None
                        new_street = update_street(tag.attrib['v'])
                        print(tag.attrib['v'], " => ", new_street)
                        tag.attrib['v'] = new_street
            
    print("\n")     
    
    # Gravar correções realizadas em um novo arquivo
    first_line = '''<?xml version="1.0"?>'''+'\n'
    root = ET.tostring(context.root).decode("utf-8")    
    new_xml = first_line + root
    f = open(output, "w", encoding="utf-8")
    f.write(new_xml)
    f.close()    
    
    osm_file.close()




if __name__ == '__main__':
    
    print("=======> Inicio cidade......: ", datetime.datetime.now(), "\n")
    osmfile = "dados/jundiai_e_regiao_map_zen.osm"
    output = "dados/output_city.osm"
    clean_city(osmfile, output)
    
    print("=======> Inicio telefone...: ", datetime.datetime.now(), "\n")    
    osmfile = "dados/output_city.osm"
    output = "dados/output_phone.osm"
    clean_phone(osmfile, output)
    
    print("=======> Inicio cód.postal.: ", datetime.datetime.now(), "\n")    
    osmfile = "dados/output_phone.osm"
    output = "dados/output_postal_code.osm"
    clean_post_code(osmfile, output)

    print("=======> Inicio ruas.......: ", datetime.datetime.now(), "\n")       
    osmfile = "dados/output_postal_code.osm"
    output = "dados/output_jundiaieregiao.osm"
    clean_street(osmfile, output)   
    
    print("=======> Fim...............: ", datetime.datetime.now())        
    

=======> Inico cidade......:  2017-12-28 18:57:22.964610 

Mairipora  =>  Mairiporã
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Várzea Paulista, SP  =>  Várzea Paulista
Jundiai  =>  Jundiaí
JUndiaí  =>  Jundiaí
JundiaÌ  =>  Jundiaí
JundiaÌ  =>  Jundiaí
Jundiái  =>  Jundiaí
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí
Campo0 Limpo Paulista  =>  Campo Limpo Paulista
Mariporã  =>  Mairiporã
Mariporã  =>  Mairiporã
Jundiai  =>  Jundiaí
Jundiai  =>  Jundiaí


=======> Inicio telefone...:  2017-12-28 18:57:36.453419
11 4038-2655  =>  +55 11 4038-2655
+55 11 45272373  =>  +55 11 4527 2373
(11) 4522-3149  =>  +55 11 4522-3149
: 11 3917-0751  =>  +55 11 3917-0751
+55 11 4531 0082;+55 11 4531 0083  =>  +55 11 4531 0082;+55 11 4531 0083
+55 11 44466767  =>  +55 11 4446 6767
+55 11 4526-1246;+55 11 4588-9446  =>  +55 11 4526-1246;+55 11 4588-9446
+55 11 45841402  =>  +55 11 4584 1402
+ 55 11 4525 5000  =>  +55 11 4525 5

Avenida dos Alpes; Avenida Pauliceia  =>  Avenida dos Alpes; Avenida Pauliceia
Avenidas das Aves Marinhas  =>  Avenida das Aves Marinhas
Rau Diogo Alveres correia  =>  Rau Diogo Alveres correia
rua Raul Breesane Malta  =>  Rua Raul Breesane Malta
Miguel De Barros  =>  Miguel De Barros
José Pereira da Silva  =>  José Pereira da Silva
Rod. Manoel Silvério Pinto  =>  Rodovia Manoel Silvério Pinto
Rua Vitoria,  =>  Rua Vitoria
Rua "R-5"  =>  Rua R-5
José Vitórino Ferreira Filho  =>  José Vitórino Ferreira Filho
Rua Manoel Pinto Rodrigues,  =>  Rua Manoel Pinto Rodrigues
Rua " 16 "  =>  Rua 16 
Pedro Barsanelli  =>  Pedro Barsanelli
Antiga Estrada da Mata Fria  =>  Antiga Estrada da Mata Fria
Ponte Velha  =>  Ponte Velha
Doutor Osvaldo Urioste  =>  Doutor Osvaldo Urioste
Complexo Viário dos Emancipadores  =>  Complexo Viário dos Emancipadores
Jean Anastace Kovelis  =>  Jean Anastace Kovelis
Rua Atílio Simoneti;Rua José Lopes  =>  Rua Atílio Simoneti;Rua José Lopes
estrada da bucolica  =>  E